In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_loc = "emotions.csv"
data = pd.read_csv(data_loc)
print(data.describe())

        # mean_0_a     mean_1_a     mean_2_a     mean_3_a     mean_4_a  \
count  2132.000000  2132.000000  2132.000000  2132.000000  2132.000000   
mean     15.256914    27.012462  -104.975629    13.605898    24.150483   
std      15.284621     9.265141   206.271960    16.874676    14.187340   
min     -61.300000  -114.000000  -970.000000  -137.000000  -217.000000   
25%       6.577500    26.075000  -195.000000     4.857500    23.600000   
50%      14.100000    30.000000    14.950000    15.400000    25.200000   
75%      27.700000    31.400000    29.600000    26.500000    26.800000   
max     304.000000    42.300000   661.000000   206.000000   213.000000   

        mean_d_0_a   mean_d_1_a   mean_d_2_a   mean_d_3_a   mean_d_4_a  ...  \
count  2132.000000  2132.000000  2132.000000  2132.000000  2132.000000  ...   
mean      0.025378     0.052282     0.301655     0.036793     0.083567  ...   
std      17.981796     8.509174    68.098894    17.010031    18.935378  ...   
min    -218.00000

In [4]:
print(data.columns)
label_mapping={'NEGATIVE':0,'NEUTRAL':1,'POSITIVE':2}

Index(['# mean_0_a', 'mean_1_a', 'mean_2_a', 'mean_3_a', 'mean_4_a',
       'mean_d_0_a', 'mean_d_1_a', 'mean_d_2_a', 'mean_d_3_a', 'mean_d_4_a',
       ...
       'fft_741_b', 'fft_742_b', 'fft_743_b', 'fft_744_b', 'fft_745_b',
       'fft_746_b', 'fft_747_b', 'fft_748_b', 'fft_749_b', 'label'],
      dtype='object', length=2549)


In [5]:
def preprocess_inputs(df):
    df=df.copy()
    
    df['label']=df['label'].replace(label_mapping)
    y=df['label'].copy()
    X=df.drop('label',axis=1).copy()
    
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=123)
    
    return X_train,X_test,y_train,y_test

In [6]:
X_train,X_test,y_train,y_test = preprocess_inputs(data)

In [7]:

inputs = tf.keras.Input(shape = (X_train.shape[1]))
hidden_layer_1 = tf.keras.layers.Dense(64, activation = 'relu')(inputs)
hidden_layer_2 = tf.keras.layers.Dense(64, activation = 'relu')(hidden_layer_1)
output = tf.keras.layers.Dense(3, activation = 'softmax')(hidden_layer_2)

model = tf.keras.Model(inputs = inputs, outputs = output)
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2548)]            0         
_________________________________________________________________
dense (Dense)                (None, 64)                163136    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 167,491
Trainable params: 167,491
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
from sklearn.metrics import confusion_matrix, classification_report
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
 )


history = model.fit(
    X_train, 
    y_train,
    batch_size = 32, 
    validation_split = 0.2,
    epochs = 10,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = 5, 
            restore_best_weights = True
        )
    ]
)


Epoch 1/10
38/38 [==============================] - 1s 8ms/step - loss: 12124685336576.0000 - accuracy: 0.5574 - val_loss: 1705151823872.0000 - val_accuracy: 0.5953
Epoch 2/10
38/38 [==============================] - 0s 5ms/step - loss: 21807597879296.0000 - accuracy: 0.4895 - val_loss: 9903520350208.0000 - val_accuracy: 0.5853
Epoch 3/10
38/38 [==============================] - 0s 5ms/step - loss: 36787334414336.0000 - accuracy: 0.6178 - val_loss: 51713419509760.0000 - val_accuracy: 0.6589
Epoch 4/10
38/38 [==============================] - 0s 5ms/step - loss: 26046353113088.0000 - accuracy: 0.6530 - val_loss: 4284101427200.0000 - val_accuracy: 0.4448
Epoch 5/10
38/38 [==============================] - 0s 5ms/step - loss: 13887664553984.0000 - accuracy: 0.6387 - val_loss: 2639178235904.0000 - val_accuracy: 0.4515
Epoch 6/10
38/38 [==============================] - 0s 5ms/step - loss: 8516544757760.0000 - accuracy: 0.6655 - val_loss: 423575617536.0000 - val_accuracy: 0.6990
Epoch 7/10


In [9]:
model_acc = model.evaluate(X_test, y_test, verbose = 0)[1]
print("Model Accuracy is ", model_acc)

Model Accuracy is  0.4468750059604645


Using a Recurrent Neural Network Instead
`

In [10]:
inputs = tf.keras.Input(shape = (X_train.shape[1]))

expand_dims = tf.expand_dims(inputs, axis = 2)
gru = tf.keras.layers.GRU(256, return_sequences = True)(expand_dims)
flatten = tf.keras.layers.Flatten()(gru)

output = tf.keras.layers.Dense(3, activation = 'softmax')(flatten)

model = tf.keras.Model(inputs = inputs, outputs = output)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2548)]            0         
_________________________________________________________________
tf.expand_dims (TFOpLambda)  (None, 2548, 1)           0         
_________________________________________________________________
gru (GRU)                    (None, 2548, 256)         198912    
_________________________________________________________________
flatten (Flatten)            (None, 652288)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 1956867   
Total params: 2,155,779
Trainable params: 2,155,779
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history=model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)